## Copy/move files from CSV files
This notebook allow to copy or move files based on two columns of a CSV file 

### Load libraries

In [ ]:
import os
import shutil
import pandas

### Settings

In [ ]:
inputdir = 'PATH_TO_INPUT_FOLDER' # Image folder, including subfolders
filetypes = set(['.png', '.jpg', '.jpeg', '.bmp']) # Define filetypes to include
outputdir = 'PATH_TO_OUTPUT_FOLDER' # Output folder
move_files = False # Move files to destination, otherwise copy the files 
csv_path = 'FULL_PATH_TO_CSV_FILE' # Path to the CSV file
csv_column1 = 'COL1' # First column of CSV containing filenames
csv_column2 = 'COL2' # Second column of CSV containing sort variable, e.g. 'Fracture'
csv_delimiter = ',' # CSV column delimiter

### Main code

In [ ]:
if not os.path.exists(inputdir) or inputdir == outputdir or not os.path.isfile(csv_path):
    print('ERROR (Directory/file issues): Please check paths.')
    exit()

# Fill dictionary from columns
df = pandas.read_csv(os.path.normpath(csv_path), dtype=str, sep=csv_delimiter, 
    usecols=[csv_column1, csv_column2]).dropna()
csv = df.set_index(csv_column1)[csv_column2].to_dict()

# Iterate over FILES in folder and subfolders
def main():
    for paths, _, files in os.walk(os.path.normpath(inputdir), topdown=True):
        for file in files:
                if any(x in file.lower() for x in filetypes) == True:
                    filestem = os.path.basename(file).split('.')[0]
                    inputfile = os.path.join(paths, file)
                    if filestem in csv and csv[filestem]:
                        outputpath = os.path.normpath(os.path.join(outputdir, csv.get(filestem, '')))
                        outputfile = os.path.join(outputpath, file)
                        if os.path.isfile(outputfile) == True: 
                            print('SKIPPED (File exists), '+inputfile)
                            continue
                        if not os.path.exists(outputpath): os.makedirs(outputpath)
                        if move_files == True: shutil.move(inputfile, outputfile)
                        else: shutil.copy2(inputfile, outputfile)
                        print('SUCCESS (File processed), ' + outputfile)
                    else:
                        print('SKIPPED (Value empty), '+ inputfile)

if __name__ == '__main__':
    main()